# Pool profiling
Comparing performance of different Pathos worker pools for counting neighbor hit segments

## Hardware info

In [ ]:
!inxi --cpu --memory --system

In [ ]:
!python -m cpuinfo

In [ ]:
import pandas as pd
from cpuinfo import get_cpu_info
pd.Series(get_cpu_info()).to_frame()

## Code version

In [ ]:
!git log | head -n3

## Run

In [ ]:
from time import time

import numpy as np
import pandas as pd
from pathos.pools import ProcessPool, ThreadPool, SerialPool
from pathos.helpers import cpu_count
import seaborn as sns

from tracking.segment import stat_seg_neighbors
from datasets import get_hits

records = []
for dataset, n_events in [('trackml_volume', 1), ('simple', 4000), ('spdsim', 2000), ('bman', 200)]:
    hits = get_hits(dataset, n_events)
    max_r = np.sqrt((hits.x.max() - hits.x.min()) ** 2 +
                    (hits.y.max() - hits.y.min()) ** 2 +
                    (hits.z.max() - hits.z.min()) ** 2)
    for pool_class in (ThreadPool, SerialPool, ProcessPool, None):
        nodecounts = (1,) if pool_class is None else range(1, cpu_count() + 1)
        for nodes in nodecounts:
            pn = 'None' if pool_class is None else pool_class.__name__
            print(dataset, pn, nodes)
            t0 = time()
            stat_seg_neighbors(hits, 0, max_r, 60, pool_class, nodes)
            t1 = time()
            records.append((dataset, pn, nodes, t1 - t0))

df = pd.DataFrame(records, columns=('dataset', 'pool', 'nodes', 't'))
df


## Processing time depending on worker count
- SerialPool does not affect performance
- ThreadPool works for large events and is counterproductive for small ones
- Processes are better than threads

In [ ]:
sns.relplot(data=df, x='nodes', y='t', col='dataset', style='pool', hue='dataset', kind='line', markers=True)

## Speedup depending on number of processes
- especially good for small events
- gets better up to the number of cores


In [ ]:
baseline = df[df.pool == 'None'].set_index('dataset').t
df['speedup'] = df.dataset.map(baseline)/df.t
pp = df[df.pool == 'ProcessPool']
max_speedup = pp.groupby('dataset', sort=False).apply(lambda g: g.iloc[g.speedup.argmax()])
sns.lineplot(data=pp, x='nodes', y='speedup', hue='dataset')
sns.scatterplot(data=max_speedup, x='nodes', y='speedup', hue='dataset', legend=False)